In [ ]:
from src import Color
from src.material.material.phong_material import PhongMaterial
from src import World
from src import Sphere, Plane
from src import PointLight, AmbientLight
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src.render import LinearRayCaster
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig

# 4 spheres in a row with different IOR (index of refraction) values
1. oil with IOR=1.2
2. glass with IOR=1.33
3. crystal with IOR=1.5
4. diamond with IOR=2.4
#### All spheres have the same transparency (0.95) and shininess (64). The ground plane is matte gray to better see the refraction effects on the HDR skybox background.

In [ ]:
# colors
brown = Color.custom_rgb(150, 75, 0)
red   = Color.custom_rgb(255, 0, 0)
white = Color.custom_rgb(245, 245, 245)
green = Color.custom_rgb(30, 180, 120)

glass_spec = Color.custom_rgb(255, 255, 255)

ior_oil = PhongMaterial(name="ior_oil", base_color=red, spec_color=glass_spec, shininess=64, ior=1.2, reflectivity=0.0, transparency=0.95)
ior_glass = PhongMaterial(name="ior_glass", base_color=red, spec_color=glass_spec, shininess=64, ior=1.33, reflectivity=0.0, transparency=0.95)
ior_crystal = PhongMaterial(name="ior_crystal", base_color=red, spec_color=glass_spec, shininess=64, ior=1.5, reflectivity=0.0, transparency=0.95)
ior_diamond = PhongMaterial(name="ior_diamond", base_color=red, spec_color=glass_spec, shininess=64, ior=2.4, reflectivity=0.0, transparency=0.95)
matte_ground = PhongMaterial(name="matte_ground", base_color=red, spec_color=Color.custom_rgb(80, 80, 80), shininess=8, reflectivity=0.0)

sphere1 = Sphere(center=Vertex(-3.5, 0.2, -5), radius=1.0, material=ior_oil)
sphere2 = Sphere(center=Vertex(-1.1, 0.2, -5), radius=1.0, material=ior_glass)
sphere3 = Sphere(center=Vertex( 1.1, 0.2, -5), radius=1.0, material=ior_crystal)
sphere4 = Sphere(center=Vertex( 3.5, 0.2, -5), radius=1.0, material=ior_diamond)

# ground plane
plane = Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0), material=matte_ground)

# --- world ---
world = World()
for obj in (sphere1, sphere2, sphere3, sphere4, plane):
    world.add(obj)

# --- lights ---
point_light_1 = PointLight(position=Vertex(5, 6, -1), intensity=1600.0, falloff=0.01)
point_light_2 = PointLight(position=Vertex(0, 5, -5), intensity=1200.0, falloff=0.01)
ambient_light = AmbientLight(intensity=0.2)
lights = [point_light_1, point_light_2, ambient_light]

# --- camera / scene ---
camera = Camera(
    fov=65,
    aspect_ratio=Resolution.R360p.aspect_ratio,
    origin=Vertex(0, 0, 0),
    direction=Vector(0, 0, -1),
)

scene = Scene(
    camera=camera,
    world=world,
    lights=lights,
    skybox_path="./skybox/shanghai_4k.hdr"
)

scene.validate()

In [ ]:
render_configuration = RenderConfig(
    resolution=Resolution.R360p,
    samples_per_pixel=1,
    max_depth=4,
)

#todo fix TQDM_BAR not working because iam not updating the progress properly in the ray tracer
preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration)
png = my_ray_tracer.render("ior.png")